In [1]:
import numpy as np
import math
import modules
import optimization
from scipy.optimize import minimize

max_iter = 2000


def ratio_finder(f):
    
    # Perturbation Size Δx Choice based on Gill et. al. 1983
    eps = 1e-6
    delta_X = math.sqrt(eps / abs(f))
    
    print('\nratio= ', "{:1.2f}".format(delta_X*100), '%')
    ratios = [1-delta_X/2, 1, 1+delta_X/2]
    return ratios

def sensitivity_sweep(v1_name, v2_name, all_vars):
    # v1: variable being swept, held constant for each optimization
    # v2: variable being optimized

    param_list, param_val, param_unit, param_bnds, param_label = optimization.default_value(v1_name)
    res_opt, op_obj = optimization.run_optimization(v2_name, [], v1_name, param_val, all_vars, max_iter)
    v1_ratios = ratio_finder(res_opt.fun)
    x_star = res_opt.x
    
    v1_list, v1_nom, v1_unit, v1_bnds, v1_label = optimization.default_value(v1_name)
    v2_list, v2_nom, v2_unit, v2_bnds, v2_label = optimization.default_value(v2_name)
    
    num_v1 = len(v1_list)
    num_v2 = len(v2_list)
    num_ratios = len(v1_ratios)
    
    v2_star = np.zeros(shape=(num_v1,num_ratios,num_v2))
    dv2_star_dv1 = np.zeros(shape=(num_v1,num_v2))

    desc = '\nFinding optimal '+ v2_label + ' while holding '+ v1_label + ' as parameter:'
    print(desc)
    #print('ratio=', v1_ratios)
    
    for i in range(num_v1):
        for j in range(num_ratios):
            v1 = dict. copy(v1_nom)
            v1[v1_list[i]] = v1_nom[v1_list[i]] * v1_ratios[j]
            print(v2_name)
            print(x_star)
            print(v1_name)
            print(v1)
            print('&&&&&&&&&&&&')
            res_opt, op_obj = optimization.run_optimization(v2_name, x_star, v1_name, v1, all_vars, max_iter)
            if res_opt.success:
                v2_star[i,j,:] = res_opt['x']
            else:
                v2_star[i,j,:] = np.nan
                print('failing for:',v1)
            #print('>>>>>',v2_star[i,j,:])
            

        idx_nom = v1_ratios.index(1)
        dv2_star = np.divide((v2_star[i,-1,:] - v2_star[i,0,:]), v2_star[i,idx_nom,:])
        dv2_star_dv1[i,:] = dv2_star / (v1_ratios[-1] - v1_ratios[0])    

    # print result in a table format
    print('\n')
    col_width = len(max(v2_list+v1_list, key=len)) + 1
    print(' '*col_width, end='')
    for i in range(num_v2):
        print(' ',v2_list[i], ' '*(col_width-len(v2_list[i])-2),end='')
    print('')
    for i in range(num_v1):
        print(v1_list[i], ' '*(col_width-len(v1_list[i])),end='|')
        for j in range(num_v2):
            if (dv2_star_dv1[i,j] == 0):
                print('  0', ' '*(col_width-4),end='|')
            elif np.isnan(dv2_star_dv1[i,j]):
                print("  \x1b[31mFailed\x1b[0m", ' '*(col_width-9),end='|')
            else:
                print("{:10.6f}".format(dv2_star_dv1[i,j]), ' '*(col_width-11),end='|')
        print('')
        
    return dv2_star_dv1


all_vars = ['x_wec','x_type_wec','x_pen', 'x_disc_pen', 'p_pen','x_env','p_wec','p_fish_salmon', 'x_es', 'p_es']


# sensitivity of optimal wec and pen design to environment/location
design_var = ['x_wec','x_pen', 'x_es']
parameter = ['x_env']
dx_wec_pen_star_dx_env = sensitivity_sweep(parameter, design_var, all_vars)
#print('\n\ndx_wec_pen_star_dx_env=\n', dx_wec_pen_star_dx_env)
#print('\n*************************************************\n\n')

# sensitivity of optimal environment/location to wec and pen design
design_var = ['x_env']
parameter = ['x_wec','x_pen', 'x_es']
dx_env_star_dx_wec_pen = sensitivity_sweep(parameter, design_var, all_vars)
#print('\n\ndx_env_star_dx_wec_pen=\n', dx_env_star_dx_wec_pen)
#print('\n*************************************************\n\n')


ratio=  0.03 %

Finding optimal x_wec & x_pen & x_es while holding x_env as parameter:


                capture_width   pen_diameter   pen_height     spacing        stock_density   pen_depth      es_size      
temp           |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
O2_in          |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
salinity       |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
U              |  0           |  0           |  0           |  0           |  0           |  0           |  0           |
wave_height    | -2.000000    | -0.000000    | -0.000000    |  0           |  0           |  0           |  0           |
wave_period    | -1.000000    |  0           | -0.000000    |  0           |  0           |  0           |  0           |


c:\Users\Arezoo_Admin\AppData\Local\Programs\miniconda3\lib\site-packages\scipy\optimize\_minimize.py:517: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x0 = np.atleast_1d(np.asarray(x0))


ValueError: setting an array element with a sequence.